# Load Grades
Reads `grades_v0.xlsx` from the path defined in `config.py` and returns a clean pandas DataFrame.

In [ ]:
import sys
sys.path.insert(0, '..')  # make src/ and config/ importable from notebooks/

from config.config import TABLE_PATH, TEMPLATE_PATH, OUTPUT_DIR
from src.xlsx_utils import get_file_info, get_sheet_names, read_sheet, get_schema, preview
from src.docx_utils import open_doc, save_doc, find_and_replace

print('Table path:    ', TABLE_PATH)
print('Template path: ', TEMPLATE_PATH)
print('Output dir:    ', OUTPUT_DIR)

In [ ]:
# Inspect the workbook before loading
info = get_file_info(TABLE_PATH)
print('Sheets found:')
for sheet_name, meta in info['sheets'].items():
    print(f"  {sheet_name!r} — {meta['rows']} rows x {meta['cols']} columns")

In [ ]:
# Load the first (and only) sheet into a DataFrame
df = read_sheet(TABLE_PATH, sheet=0)

print(f'Loaded {len(df)} students, {len(df.columns)} columns')
df

In [ ]:
# Column schema: dtypes and null check
get_schema(df)

In [ ]:
# Quick preview of the first 5 rows
preview(df)

## Iterate over students
Loop over every row and unpack each column into its own variable.

In [ ]:
# Make sure the output folder exists
OUTPUT_DIR.mkdir(parents=True, exist_ok=True)

# Stem of the template filename, used to build each output filename
template_stem = TEMPLATE_PATH.stem   # e.g. "informe 4 anys FEBRER curs 25-26"

for _, row in df.iterrows():

    # --- Identity ---
    student                = row['Estudiant']

    # --- Language / Writing ---
    writes_own_name        = row['Escriu el seu nom sense model']
    phonics                = row['(vocals + P L M D G F C)']
    copies_from_board      = row['Copia paraules de la pissarra']
    dictation              = row['Realitza dictat lletra a lletra']

    # --- Reading & Communication ---
    interest_in_reading    = row['Mostra interès per la lectura de literatura infantil']
    oral_communication     = row['Participa en situacions de comunicació oral']

    # --- Technology ---
    use_of_devices         = row['Fa un bon us dels dispositius tecnologics (pissarra, tablet, blue-bot)']

    # --- Arts ---
    james_rizzi            = row['James Rizzi']
    drawing                = row["Realitza dibuixos adequats a l'edat"]
    use_of_tools           = row['Utilitza correctament les diferents eines (pinzells, tisores, etc.)']

    # -----------------------------------------------------------------------
    # 1. Open a fresh copy of the template for this student
    # -----------------------------------------------------------------------
    doc = open_doc(str(TEMPLATE_PATH))

    # -----------------------------------------------------------------------
    # 2. Fill in the student name placeholder
    # -----------------------------------------------------------------------
    find_and_replace(doc, 'NOM: ', f'NOM: {student}')

    # -----------------------------------------------------------------------
    # 3. Save to OUTPUT_DIR, named "<template_stem>_<student>.docx"
    # -----------------------------------------------------------------------
    output_path = OUTPUT_DIR / f"{template_stem}_{student}.docx"
    save_doc(doc, str(output_path))

    print(f"Saved: {output_path.name}")